## Case Python - Jump Start - Turma 04 - Análise IMDb
### Marcos Gonçalves

- [IMDB](https://www.imdb.com/) (Internet Movie Database) é uma base de dados online para avaliação e informações sobre filmes e séries de TV.

<center>
    
<p><img src="https://m.media-amazon.com/images/G/01/IMDb/brand/guidelines/imdb/IMDb_BrandBanner_1920x425.jpg" alt="Logo_IMDb" width='85%'></p>

</center>

## Bases Utilizadas:

- title basics
- title ratings
- title akas  
##### Disponível em: [IMDB](https://datasets.imdbws.com/)

## Dicionário dos dados

**title.basics**    
- tconst (string) - identificador alfanumérico exclusivo do título
- titleType (string) – o tipo/formato do título (por exemplo, filme, curta, série de TV, episódio de TV, vídeo, etc.)
- primaryTitle (string) – o título mais popular / o título usado pelos cineastas em materiais promocionais no momento do lançamento
- originalTitle (string) - título original, no idioma original
- isAdult (booleano) - 0: título não adulto; 1: título adulto
- startYear (YYYY) – representa o ano de lançamento de um título. No caso de séries de TV, é o ano de início da série
- endYear (YYYY) – Ano final da série de TV. '\N' para todos os outros tipos de títulos
- runtimeMinutes – tempo de execução principal do título, em minutos
- gêneros (matriz de strings) – inclui até três gêneros associados ao título

**title.ratings**    
- tconst (string) - identificador alfanumérico exclusivo do título
- averageRating – média ponderada de todas as avaliações individuais dos usuários
- numVotes - número de votos que o título recebeu


**título.akas**    
- titleId (string) - um tconst, um identificador alfanumérico exclusivo do título
- ordering (integer) – um número para identificar exclusivamente as linhas para um determinado titleId
- title (string)  – o título localizado
- region (string) - a região para esta versão do título
- language (string)- o idioma do título
- types (array) - Conjunto enumerado de atributos para este título alternativo. Um ou mais dos seguintes: "alternative", "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay". Novos valores podem ser adicionados no futuro sem aviso
- attributes (array) - Termos adicionais para descrever este título alternativo, não enumerados
- isOriginalTitle (boolean) – 0: título não original; 1: título original


## Importando Biblioteca

In [1]:
import pandas as pd

## Carregando os Arquivos

In [2]:
#title_basics
chunk_size = 50000
chunkstb = []

try:
    for chunk in pd.read_csv(r'C:\Users\marcosgoncalves\Desktop\Dados_Programacao\JumpStart2024\Python\Case_Python\title.basics.tsv', sep= '\t', chunksize=chunk_size):
        chunkstb.append(chunk)
    df_title_basics = pd.concat(chunkstb, axis=0)
except pd.errors.ParserError as e:
    print(f'Erro ao ler o arquivo')

In [3]:
#title_ratings
df_title_ratings = pd.read_csv(r'C:\Users\marcosgoncalves\Desktop\Dados_Programacao\JumpStart2024\Python\Case_Python\title.ratings.tsv', sep= '\t')

In [2]:
#title_akas
chunk_size = 50000
chunksta = []

try:
    for chunk in pd.read_csv(r'C:\Users\marcosgoncalves\Desktop\Dados_Programacao\JumpStart2024\Python\Case_Python\title.akas.tsv', sep= '\t', chunksize=chunk_size):
        chunksta.append(chunk)
    df_title_akas = pd.concat(chunksta, axis=0)
except pd.errors.ParserError as e:
    print(f'Erro ao ler o arquivo')

## Análise Exploratória e tratamento dos dados

#### Volume e tipo de dados Title Basics

A base total possui 10.918.941 linhas e 9 colunas

In [4]:
df_title_basics.shape

(10918941, 9)

In [5]:
df_title_basics['titleType'].unique()

array(['short', 'movie', 'tvShort', 'tvMovie', 'tvEpisode', 'tvSeries',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'tvPilot'],
      dtype=object)

In [6]:
df_title_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10918941 entries, 0 to 10918940
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 749.7+ MB


In [7]:
df_title_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


#### Aplicando um filtro para trabalhar apenas com filmes identificados por movie, tvMovie, short e tvShort, descartando as demais categorias que não se tratam de filmes

#### Reduzindo a base para 1.848.568 linhas e mantendo as 9 colunas

In [8]:
df_title_basics_filmes = df_title_basics[(df_title_basics['titleType'] == 'movie') | (df_title_basics['titleType'] =='tvMovie') 
|(df_title_basics['titleType'] =='short') | (df_title_basics['titleType'] =='tvShort')]

In [9]:
df_title_basics_filmes.shape

(1848568, 9)

#### Os valores nulos estavam com a informação "\N" foi corrigida para nulo para ser possível visualizar as quantidades de nulos no DF

In [10]:
df_title_basics_filmes = df_title_basics_filmes.replace(r'\N', None)

#### Ajustes nos tipos de dados
- Ano estréia e ano encerramento alterado para formato data
- Tempo de duração alterado para float
- is Adulto para booleano

In [11]:
df_title_basics_filmes[['startYear', 'endYear']] = df_title_basics_filmes[['startYear', 'endYear']].apply(pd.to_datetime, format='%Y')
df_title_basics_filmes['runtimeMinutes'] = df_title_basics_filmes['runtimeMinutes'].astype(float)
df_title_basics_filmes['isAdult'] = df_title_basics_filmes['isAdult'].astype(bool)

In [12]:
df_title_basics_filmes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1848568 entries, 0 to 10918939
Data columns (total 9 columns):
 #   Column          Dtype         
---  ------          -----         
 0   tconst          object        
 1   titleType       object        
 2   primaryTitle    object        
 3   originalTitle   object        
 4   isAdult         bool          
 5   startYear       datetime64[ns]
 6   endYear         datetime64[ns]
 7   runtimeMinutes  float64       
 8   genres          object        
dtypes: bool(1), datetime64[ns](2), float64(1), object(5)
memory usage: 128.7+ MB


#### Verificando a contagem de dados para cada coluna, 'endYear' que está com 0, ou seja, todos dados nulos, será excluida da base

In [13]:
df_title_basics_filmes.count()

tconst            1848568
titleType         1848568
primaryTitle      1848563
originalTitle     1848563
isAdult           1848568
startYear         1707289
endYear                 0
runtimeMinutes    1186208
genres            1759593
dtype: int64

In [14]:
df_title_basics_filmes.drop(columns=['endYear'], inplace=True)

### Volume e tipo de dados Title Ratings

In [15]:
df_title_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454745 entries, 0 to 1454744
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1454745 non-null  object 
 1   averageRating  1454745 non-null  float64
 2   numVotes       1454745 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 33.3+ MB


- **A base total possui 1.454.745 linhas e 3 colunas**
- Não possui valores nulos

In [16]:
df_title_ratings.shape

(1454745, 3)

In [17]:
df_title_ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2063
1,tt0000002,5.6,279
2,tt0000003,6.5,2030
3,tt0000004,5.4,180
4,tt0000005,6.2,2796


### Volume e tipo de dados Title Akas

In [4]:
df_title_akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48911203 entries, 0 to 48911202
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  int64 
dtypes: int64(2), object(6)
memory usage: 2.9+ GB


In [5]:
df_title_akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita,\N,\N,original,\N,1
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita,US,\N,imdbDisplay,\N,0
3,tt0000001,4,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
4,tt0000001,5,Καρμενσίτα,GR,\N,imdbDisplay,\N,0


### Join/Merge para unir as avaliações e votos no df de filmes

#### Verificando o nome e tipo da coluna que será usada para fazer a conexão

In [18]:
df_title_basics_filmes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1848568 entries, 0 to 10918939
Data columns (total 8 columns):
 #   Column          Dtype         
---  ------          -----         
 0   tconst          object        
 1   titleType       object        
 2   primaryTitle    object        
 3   originalTitle   object        
 4   isAdult         bool          
 5   startYear       datetime64[ns]
 6   runtimeMinutes  float64       
 7   genres          object        
dtypes: bool(1), datetime64[ns](1), float64(1), object(5)
memory usage: 114.6+ MB


In [19]:
df_title_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454745 entries, 0 to 1454744
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1454745 non-null  object 
 1   averageRating  1454745 non-null  float64
 2   numVotes       1454745 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 33.3+ MB


In [15]:
df_filmes_avaliacoes = pd.merge(
    df_title_basics_filmes,
    df_title_ratings,
    on=['tconst'],
    how='left'
)

#### Possível notar que tem 1.319.489 dados nulos nas colunas de média de avaliações (averageRating) e para Numero de votos (numVotes)
#### Foi entendido que os filmes que estão sem avaliações ou votos, não receberam nenhuma avaliação ou votos na plataforma

In [21]:
df_filmes_avaliacoes.isnull().sum()

tconst                  0
titleType               0
primaryTitle            5
originalTitle           5
isAdult                 0
startYear          141279
runtimeMinutes     662360
genres              88975
averageRating     1319489
numVotes          1319489
dtype: int64

#### Dropar os dados nulos para não afetar a análise

In [11]:
df_filmes_avaliacoes = df_filmes_avaliacoes.dropna(subset=['averageRating','startYear','runtimeMinutes','genres'])

In [12]:
df_filmes_avaliacoes.isnull().sum()

tconst            0
titleType         0
primaryTitle      0
originalTitle     0
isAdult           0
startYear         0
runtimeMinutes    0
genres            0
averageRating     0
numVotes          0
dtype: int64

In [13]:
df_filmes_avaliacoes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 463632 entries, 0 to 1848563
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   tconst          463632 non-null  object        
 1   titleType       463632 non-null  object        
 2   primaryTitle    463632 non-null  object        
 3   originalTitle   463632 non-null  object        
 4   isAdult         463632 non-null  bool          
 5   startYear       463632 non-null  datetime64[ns]
 6   runtimeMinutes  463632 non-null  float64       
 7   genres          463632 non-null  object        
 8   averageRating   463632 non-null  float64       
 9   numVotes        463632 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(3), object(5)
memory usage: 35.8+ MB


#### Após retirar os valores nulos o df_filmes_avaliacoes será usado para ajuste dos generos

In [25]:
df_filmes_avaliacoes.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,False,1894-01-01,1.0,"Documentary,Short",5.7,2063.0
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,False,1892-01-01,5.0,"Animation,Short",5.6,279.0
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,False,1892-01-01,5.0,"Animation,Comedy,Romance",6.5,2030.0
3,tt0000004,short,Un bon bock,Un bon bock,False,1892-01-01,12.0,"Animation,Short",5.4,180.0
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,False,1893-01-01,1.0,"Comedy,Short",6.2,2796.0


#### Criado um novo DataFrame pois os generos estão juntos. Fazendo a separação para que as análises por generos seja mais assertivas

- Fazendo uma cópia do df_filmes_avaliações que possui as informações incluindo as avaliações

In [14]:
contagem_genero = df_filmes_avaliacoes.copy()

- Duplicando a coluna de generos para uma delas ser subdividida pelos filmes que possui mais de um genero

In [15]:
contagem_genero = contagem_genero.assign(genres_sub=contagem_genero['genres'])

In [16]:
contagem_genero.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,genres_sub
0,tt0000001,short,Carmencita,Carmencita,False,1894-01-01,1.0,"Documentary,Short",5.7,2063.0,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,False,1892-01-01,5.0,"Animation,Short",5.6,279.0,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,False,1892-01-01,5.0,"Animation,Comedy,Romance",6.5,2030.0,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,False,1892-01-01,12.0,"Animation,Short",5.4,180.0,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,False,1893-01-01,1.0,"Comedy,Short",6.2,2796.0,"Comedy,Short"


- De fato fazendo a separação dos filmes que possuem mais de um genero

- Utilizado o método .explode no qual se passsa uma coluna como parametro para expandir  mantendo as demais colunas do df inalteradas
- Utilizado para manter a informação dos generos de forma separadas mas permanecer o titulo e demais informações sobre o filme

In [17]:
contagem_genero = contagem_genero.assign(genres=contagem_genero['genres'].str.split(',')).explode('genres').reset_index(drop=True)

In [18]:
contagem_genero.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,genres_sub
0,tt0000001,short,Carmencita,Carmencita,False,1894-01-01,1.0,Documentary,5.7,2063.0,"Documentary,Short"
1,tt0000001,short,Carmencita,Carmencita,False,1894-01-01,1.0,Short,5.7,2063.0,"Documentary,Short"
2,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,False,1892-01-01,5.0,Animation,5.6,279.0,"Animation,Short"
3,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,False,1892-01-01,5.0,Short,5.6,279.0,"Animation,Short"
4,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,False,1892-01-01,5.0,Animation,6.5,2030.0,"Animation,Comedy,Romance"


## Questões de Negócio

### 1. Quais são as categorias de filmes mais comuns no IMDB?

- Analisando as categorias movie, short, tvMovie e tvShort, a categoria que mais possui titulos é a de movie

In [31]:
df_filmes_avaliacoes.head(3)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,False,1894-01-01,1.0,"Documentary,Short",5.7,2063.0
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,False,1892-01-01,5.0,"Animation,Short",5.6,279.0
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,False,1892-01-01,5.0,"Animation,Comedy,Romance",6.5,2030.0


In [32]:
categorias = df_filmes_avaliacoes['titleType'].value_counts()
categorias = categorias.reset_index()
categorias.columns = ['Categoria','Quantidade_Titulos']

In [33]:
categorias.head()

,Categoria,Quantidade_Titulos
0,movie,276247
1,short,141269
2,tvMovie,43988
3,tvShort,2128


### 2. Qual o número de títulos por gênero?

- Utilizado o df contagem_genero que possui os generos separados

In [34]:
contagem_titulos_genero = contagem_genero['genres'].value_counts()
contagem_titulos_genero = pd.DataFrame(contagem_titulos_genero.reset_index())
contagem_titulos_genero.columns = ['Genero', 'Quantidade']

In [35]:
contagem_titulos_genero

,Genero,Quantidade
0,Drama,191550
1,Short,126642
2,Comedy,114001
3,Documentary,74577
4,Romance,42042
5,Action,36142
6,Horror,33858
7,Crime,32837
8,Thriller,32517
9,Animation,28478


### 3. Qual a mediana de avaliação dos filmes Por Gênero?

In [36]:
mediana = contagem_genero.groupby('genres')['averageRating'].median().reset_index()
mediana = mediana.sort_values(by='averageRating',ascending=False).reset_index(drop=True)
mediana.columns = ['Genero', 'Mediana_Avaliacao']

In [37]:
mediana

,Genero,Mediana_Avaliacao
0,News,7.20
1,Documentary,7.20
2,Biography,7.00
3,History,7.00
4,Music,6.90
5,Reality-TV,6.85
6,Sport,6.80
7,Talk-Show,6.80
8,Short,6.80
9,Game-Show,6.75


### 4. Qual a mediana de avaliação dos filmes em relação ao ano de estreia?

In [38]:
ano_estreia = df_filmes_avaliacoes.copy()

In [39]:
ano_estreia['Ano'] = ano_estreia['startYear'].dt.year

In [40]:
ano_estreia = ano_estreia.groupby('Ano')['averageRating'].median().reset_index()
ano_estreia = ano_estreia.sort_values(by='averageRating', ascending=False).reset_index(drop=True)
ano_estreia.columns = ['Ano_Estreia', 'Mediana_Avaliacao']

In [41]:
ano_estreia

,Ano_Estreia,Mediana_Avaliacao
0,2024,7.00
1,1874,6.80
2,2004,6.70
3,2014,6.70
4,2005,6.70
...,...,...
139,1890,4.85
140,1893,4.80
141,1903,4.80
142,1904,4.80


### 5. Qual o número de filmes avaliados por gênero em relação ao ano de estreia?

In [42]:
filmes_ano_estreia = contagem_genero.copy()
filmes_ano_estreia['Ano'] = filmes_ano_estreia['startYear'].dt.year
filmes_ano_estreia.head(3)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,genres_sub,Ano
0,tt0000001,short,Carmencita,Carmencita,False,1894-01-01,1.0,Documentary,5.7,2063.0,"Documentary,Short",1894
1,tt0000001,short,Carmencita,Carmencita,False,1894-01-01,1.0,Short,5.7,2063.0,"Documentary,Short",1894
2,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,False,1892-01-01,5.0,Animation,5.6,279.0,"Animation,Short",1892


In [43]:
filmes_ano_estreia = filmes_ano_estreia.groupby(['genres','Ano']).size().reset_index(name='Numero_filmes')
filmes_ano_estreia = filmes_ano_estreia.sort_values(by=['Numero_filmes','Ano'],ascending=False).reset_index(drop=True)
filmes_ano_estreia = filmes_ano_estreia.rename(columns={'genres': 'Genero'})

In [44]:
filmes_ano_estreia

,Genero,Ano,Numero_filmes
0,Drama,2016,7044
1,Drama,2017,7040
2,Drama,2013,6998
3,Drama,2014,6950
4,Drama,2015,6949
...,...,...,...
3007,Documentary,1881,1
3008,Documentary,1878,1
3009,Sport,1878,1
3010,Documentary,1874,1


### 6. Qual o filme com maior tempo de duração? Calcule os percentis considerando o tempo de duração.

- Abaixo os percentis e o filme com maior tempo, se trata de uma Animação com o nome de 100, estreada em 2019 com um total de 991 horas

In [45]:
df_title_basics_filmes['runtimeMinutes'].describe()

count    1.186208e+06
mean     4.589408e+01
std      9.904545e+01
min      0.000000e+00
25%      1.000000e+01
50%      2.600000e+01
75%      8.400000e+01
max      5.946000e+04
Name: runtimeMinutes, dtype: float64

In [46]:
df_title_basics_filmes[df_title_basics_filmes['runtimeMinutes'] == 59460]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
7049083,tt29302558,movie,100,100,False,2019-01-01,59460.0,Animation


In [47]:
horas = 59460 / 60
horas

991.0

### 7. Qual a relação entre duração e gênero?

In [48]:
contagem_genero.head(3)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,genres_sub
0,tt0000001,short,Carmencita,Carmencita,False,1894-01-01,1.0,Documentary,5.7,2063.0,"Documentary,Short"
1,tt0000001,short,Carmencita,Carmencita,False,1894-01-01,1.0,Short,5.7,2063.0,"Documentary,Short"
2,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,False,1892-01-01,5.0,Animation,5.6,279.0,"Animation,Short"


In [49]:
duracao_genero = contagem_genero.groupby(['genres'])['runtimeMinutes'].mean().reset_index()
duracao_genero = duracao_genero.rename(columns = {'genres': 'genero', 'runtimeMinutes': 'duracao_média'})
duracao_genero = duracao_genero.sort_values(by=['duracao_média'], ascending=False).reset_index(drop=True)
duracao_genero

,genero,duracao_média
0,Action,88.736346
1,War,87.847977
2,Crime,87.421324
3,Romance,86.730722
4,History,86.549358
5,Musical,85.015839
6,Thriller,84.122982
7,Biography,83.799170
8,Film-Noir,81.813288
9,Adventure,81.328555


### 8. Qual é a relação entre o orçamento dos filmes e sua avaliação? 

- Questão Anulada por não conter informações de orçamento nas bases do IMDb

### 9. Qual o número de filmes produzidos por país?

- Não foi possível carregar a base por conta da memória do computador

- Mesmo carregando o arquivo em partes, não foi possível realizar as análises na base df_title_akas por conta de falta de memória e processamento do computador (Base com 48.911.203 linhas e 8 colunas)

In [ ]:
df_filmes_pais = df_title_akas[(df_title_akas['isOriginalTitle'] == 1) | (df_title_akas['region'] != r'\N')]

### 10. Quais são os top 15 melhores filmes e 15 piores filmes?

- Classificado as maiores votações como critério de desempate para as notas iguais
- Quanto mais votos e melhores avaliações, considerado melhor

In [50]:
df_filmes_avaliacoes['averageRating'].describe()

count    463632.000000
mean          6.400977
std           1.375024
min           1.000000
25%           5.600000
50%           6.500000
75%           7.300000
max          10.000000
Name: averageRating, dtype: float64

In [33]:
top_filmes = df_filmes_avaliacoes.sort_values(by=['averageRating', 'numVotes'], ascending=False)
top_filmes.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
1231315,tt30151030,short,Take 69 B,Take 69 B,False,2020-01-01,17.0,"Drama,Mystery,Romance",10.0,206.0
1233028,tt30219905,short,Yugo: The Non-Game,Yugo: The Non-Game,False,2023-01-01,20.0,Short,10.0,120.0
1835027,tt9646734,short,Dying Out Loud,Dying Out Loud,False,2019-01-01,20.0,"Documentary,Short",10.0,77.0
549754,tt12137414,short,Marty's Calling,Marty's Calling,False,2020-01-01,10.0,"Drama,Short",10.0,60.0
966343,tt21139706,movie,Prince Oak Oakleyski and News Interviewers,Prince Oak Oakleyski and News Interviewers,False,2022-01-01,60.0,"Documentary,News,Reality-TV",10.0,58.0


In [34]:
top_15_melhores = top_filmes.head(15).reset_index()
top_15_melhores = top_15_melhores.rename(columns={'originalTitle':'Titulo_Original', 'averageRating': 'Média_Avaliação','numVotes': 'Total_Votos'})
top_15_melhores[['Titulo_Original','Média_Avaliação','Total_Votos']]

,Titulo_Original,Média_Avaliação,Total_Votos
0,Take 69 B,10.0,206.0
1,Yugo: The Non-Game,10.0,120.0
2,Dying Out Loud,10.0,77.0
3,Marty's Calling,10.0,60.0
4,Prince Oak Oakleyski and News Interviewers,10.0,58.0
5,Duma about Life (after Death),10.0,47.0
6,100% Happiness,10.0,43.0
7,Yasenova,10.0,40.0
8,Beyond Ink: Deep Water (Pilot),10.0,39.0
9,Marasim - Beyond Borders,10.0,38.0


- Classificado as maiores votações como critério de desempate para as notas iguais
- Quanto mais votos e piores avaliações, considerado pior

In [35]:
top_piores_filmes = top_filmes.sort_values(by=['averageRating','numVotes'], ascending=[True,False])
top_piores_filmes.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
1589713,tt5988370,movie,Reis,Reis,False,2017-01-01,108.0,"Biography,Drama",1.0,74095.0
1685706,tt7221896,movie,Cumali Ceber: Allah Seni Alsin,Cumali Ceber: Allah Seni Alsin,False,2017-01-01,100.0,Comedy,1.0,39408.0
631272,tt13423846,movie,321 Action,321 Action,False,2020-01-01,100.0,Drama,1.0,10178.0
582244,tt12664876,short,Keratin,Keratin,False,2020-01-01,7.0,"Drama,Horror,Short",1.0,2833.0
657874,tt13788842,movie,2025 - The World enslaved by a Virus,2025: The World Enslaved by a Virus,False,2021-01-01,92.0,"Adventure,Sci-Fi",1.0,2549.0


In [36]:
top_15_piores = top_piores_filmes.head(15).reset_index()
top_15_piores = top_15_piores.rename(columns={'originalTitle':'Titulo_Original', 'averageRating': 'Média_Avaliação','numVotes': 'Total_Votos'})
top_15_piores[['Titulo_Original','Média_Avaliação','Total_Votos']]

,Titulo_Original,Média_Avaliação,Total_Votos
0,Reis,1.0,74095.0
1,Cumali Ceber: Allah Seni Alsin,1.0,39408.0
2,321 Action,1.0,10178.0
3,Keratin,1.0,2833.0
4,2025: The World Enslaved by a Virus,1.0,2549.0
5,Romeo & Juliet,1.0,2404.0
6,Nyay: The Justice,1.0,1672.0
7,Holnap történt - A nagy bulvárfilm,1.0,966.0
8,Laz Kit,1.0,860.0
9,Cumali Ceber 666,1.0,839.0


### 11. Quais são os gêneros mais populares em cada década?

In [132]:
populares_decada = contagem_genero.copy()

- Criando uma coluna decada e aplicando a função para que converta os anos em decadas

- Como busca apenas as decadas por exemplo, 1980, 1990, etc.
- Aplicado na coluna do ano de estréia, divisão inteira por 10 e multiplicado por 10 - Ex. 1878 / 10 = 187,80 (parte inteira 187 * 10 = 1870)

In [133]:
populares_decada['Decada'] = populares_decada['startYear'].apply(lambda x: (x.year // 10) * 10)

- Agrupando os generos por cada década, fazendo uma contagem para verificar quais mais apareciam em cada uma das décadas

In [134]:
generos_decada = populares_decada.groupby('genres')['Decada'].value_counts().reset_index()
generos_decada = generos_decada.rename(columns={'genres':'Genero', 'count': 'Contagem_Filmes'})
generos_decada

,Genero,Decada,Contagem_Filmes
0,Action,2010,12192
1,Action,2000,5683
2,Action,1990,4050
3,Action,2020,3966
4,Action,1980,3510
...,...,...,...
355,Western,2020,190
356,Western,1990,169
357,Western,1980,143
358,Western,1900,18


- Criar um ranking dos melhores em cada década

In [135]:
ranking = generos_decada.sort_values(by=['Decada','Contagem_Filmes'],ascending=False).reset_index(drop=True)

In [59]:
ranking['Decada'].unique()

array([2020, 2010, 2000, 1990, 1980, 1970, 1960, 1950, 1940, 1930, 1920,
       1910, 1900, 1890, 1880, 1870], dtype=int64)

In [60]:
ranking['Classificacao'] = ranking.groupby('Decada').cumcount() + 1

In [61]:
ranking.head()

,Genero,Decada,Contagem_Filmes,Classificacao
0,Drama,2020,20616,1
1,Short,2020,13725,2
2,Documentary,2020,11539,3
3,Comedy,2020,10563,4
4,Horror,2020,6029,5


- Top 3 para cada década

In [62]:
ranking[ranking['Classificacao'] <=3]

,Genero,Decada,Contagem_Filmes,Classificacao
0,Drama,2020,20616,1
1,Short,2020,13725,2
2,Documentary,2020,11539,3
27,Drama,2010,66641,1
28,Short,2010,51284,2
29,Comedy,2010,35847,3
54,Drama,2000,32798,1
55,Short,2000,28404,2
56,Comedy,2000,20411,3
81,Drama,1990,15472,1


- Top 1 para cada decada

In [63]:
ranking[ranking['Classificacao'] ==1]

,Genero,Decada,Contagem_Filmes,Classificacao
0,Drama,2020,20616,1
27,Drama,2010,66641,1
54,Drama,2000,32798,1
81,Drama,1990,15472,1
107,Drama,1980,13478,1
133,Drama,1970,12250,1
159,Drama,1960,9693,1
185,Drama,1950,7011,1
210,Drama,1940,4705,1
235,Comedy,1930,5166,1


### 12. Faça uma análise dos filmes e mostre para o investidor uma análise gerencial dos filmes no dataset (análise livre para mostrar sua criatividade, desenvolvimento e habilidades do Python).

- Analisando os generos dos filmes
- Drama é o genero com maior número de votos, foi por várias décadas o que teve maior quantidade de filmes e sua mediana de avaliação está entre os primeiros generos.

In [154]:
contagem_genero.groupby('genres')['numVotes'].sum().sort_values(ascending=False).reset_index()

,genres,numVotes
0,Drama,585771596.0
1,Action,382473291.0
2,Comedy,351629778.0
3,Adventure,301693723.0
4,Crime,220942297.0
5,Thriller,192211786.0
6,Sci-Fi,142250630.0
7,Romance,141115061.0
8,Mystery,122245108.0
9,Horror,113499313.0


In [151]:
analise_genero = contagem_genero.copy()

In [152]:
analise_genero = analise_genero.groupby('genres')['averageRating'].median().reset_index()
analise_genero = analise_genero.sort_values(by='averageRating',ascending=False).reset_index(drop=True)
analise_genero.columns = ['Genero', 'Mediana_Avaliacao']

In [153]:
analise_genero.head(15)

,Genero,Mediana_Avaliacao
0,News,7.20
1,Documentary,7.20
2,Biography,7.00
3,History,7.00
4,Music,6.90
5,Reality-TV,6.85
6,Sport,6.80
7,Talk-Show,6.80
8,Short,6.80
9,Game-Show,6.75
